In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.0 MB/s eta 0:00:00


In [ ]:
!pip install together

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.5 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.13
    Uninstalling pydantic-1.10.13:
      Successfully uninstalled pydantic-1.10.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not install

In [ ]:
import datasets
import together
import json
import tqdm


### Exlpore if the model api is able to generate different answers for different runs/temperature settings

In [ ]:
together.api_key = ""


def togther_api(prompt, option = None, verbose = False, model = "togethercomputer/llama-2-13b-chat"):
    if option == None:
        prompt = "<human>:" + prompt
    else:
        prompt = "<human>:" + prompt + "\n<bot>: The correct answer is " + option + "\n<human> Q: Why is that?"

    if verbose:
        print (prompt)

    output = together.Complete.create(
        prompt=prompt,
        model=model,
        max_tokens=1024,
        temperature=0.7,
        top_k=50,
        top_p=0.7,
        repetition_penalty=1,
        stop=['<human>', '\n\n']
    )
    return (output['output']['choices'][0]['text'])

In [ ]:
medhalt_raw = datasets.load_dataset('MedHALT/Med-HALT', 'reasoning_FCT')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
medhalt_raw["train"]

Dataset({
    features: ['id', 'dataset', 'question', 'options', 'correct_answer', 'correct_index', 'split_type', 'subject_name', 'topic_name', 'year', 'exam_name', 'student_answer', 'student_index'],
    num_rows: 18866
})

In [ ]:
q = medhalt_raw["train"]["question"][0]
medhalt_raw["train"]["question"][0]

'Which of the following structural elements is characteristic of the ortopramide group drugs?'

## Exploring few-shots

In [ ]:
together.api_key = ""


def togther_api_few_shots(prompt, option = None, verbose = False, model = "togethercomputer/llama-2-13b-chat"):
    if option == None:
        prompt = "\n####human:" + prompt
    else:
        prompt = "\n####human:" + prompt + "\n####assistant: The correct answer is " + option + "\n<human> Q: Why is that?"

    if verbose:
        print (prompt)

    output = together.Complete.create(
        prompt=prompt,
        model=model,
        max_tokens=1024,
        temperature=0.7,
        top_k=50,
        top_p=0.7,
        repetition_penalty=1,
        stop=['<human>', '\n\n']
    )
    return (output['output']['choices'][0]['text'])

In [ ]:
op = eval(medhalt_raw["train"]["options"][0])
op

{'0': 'They are anilides with propyl group in ortho.',
 '1': 'They are benzamides with methoxy group in ortho.',
 '2': 'They are benzenesulfonamides with a methyl group in ortho.',
 '3': 'They are ortho-halogenated derivatives of phenothiazine.',
 'correct answer': 'They are ortho-halogenated derivatives of phenothiazine.'}

In [ ]:
cor_ans = op["correct answer"]

In [ ]:
cor_ans

'They are ortho-halogenated derivatives of phenothiazine.'

In [ ]:
q1 = q + " ".join(['A', 'B', 'C', 'D'][i] + " " + list(op.values())[i] for i in range(4))
q1

'Which of the following structural elements is characteristic of the ortopramide group drugs?A They are anilides with propyl group in ortho. B They are benzamides with methoxy group in ortho. C They are benzenesulfonamides with a methyl group in ortho. D They are ortho-halogenated derivatives of phenothiazine.'

In [ ]:
q2 = q + " ".join(['A', 'B', 'C', 'D'][i] + " " + list(op.values())[3-i] for i in range(4))
q2

'Which of the following structural elements is characteristic of the ortopramide group drugs?A They are ortho-halogenated derivatives of phenothiazine. B They are benzenesulfonamides with a methyl group in ortho. C They are benzamides with methoxy group in ortho. D They are anilides with propyl group in ortho.'

In [ ]:
prompt = q2

In [ ]:
model = "togethercomputer/llama-2-13b-chat"
output = together.Complete.create(
    prompt=prompt,
    model=model,
    max_tokens=1024,
    temperature=0.7,
    top_k=50,
    top_p=0.7,
    repetition_penalty=1,
    stop=['<human>', '\n\n']
)

In [ ]:
output['output']['choices'][0]['text']

'\nWhich of the following structural elements is characteristic of the ortopramide group drugs?\nA They are anilides with propyl group in ortho.\nB They are benzamides with methoxy group in ortho.\nC They are benzenesulfonamides with a methyl group in ortho.\nD They are ortho-halogenated derivatives of phenothiazine.\nCorrect Answer: B\nThe correct answer is B. The ortopramide group drugs are characterized by the presence of a benzamide group with a methoxy group in the ortho position.\n\n'

In [ ]:
cor_ans

'They are ortho-halogenated derivatives of phenothiazine.'

# Exploring metrics for fitlering generated data

In [ ]:
!pip install rouge
from rouge import Rouge

Cosine Similarity

In [ ]:
# cosine similarity
from transformers import BertTokenizer, BertModel
import torch
from torch.nn.functional import cosine_similarity
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model_cs = BertModel.from_pretrained('bert-base-uncased')

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def cosine(sentence1, sentence2):
  inputs1 = tokenizer(sentence1, return_tensors='pt')
  inputs2 = tokenizer(sentence2, return_tensors='pt')
  with torch.no_grad():
    outputs1 = model_cs(**inputs1)
    outputs2 = model_cs(**inputs2)
    sentence_embedding1 = mean_pooling(outputs1, inputs1['attention_mask'])
    sentence_embedding2 = mean_pooling(outputs2, inputs2['attention_mask'])
    cosine_sim = cosine_similarity(sentence_embedding1, sentence_embedding2)
  return (cosine_sim.item())


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Rouge

In [ ]:
rouge = Rouge()

NSP

In [ ]:
from transformers import BertTokenizer, BertForNextSentencePrediction
import torch
model_nsp = BertForNextSentencePrediction.from_pretrained('bert-base-uncased') # this model has a head on top of the CLS token for NSP


def nsp(q, a):
  prompt = "[CLS]" + q + "[SEP]" + a + "[SEP]"
  input = tokenizer(prompt, return_tensors="pt")
  with torch.no_grad():
    logits = model_nsp(**input).logits
    soft = torch.nn.functional.softmax(logits, dim = 1)
    nsp_score = soft[0][0].item()
  return nsp_score

Start

In [ ]:
with open("medhalt_corrected.json", "r") as file:
  data = json.load(file)

In [ ]:
i = data[3]
i

{'question': '‘Blackouts’ in a person of alcohol consumption is seen in which of the following conditions?',
 'correct_answer': 'Alcohol intoxication',
 'options': ['A)Alcohol dependence',
  'B)Alcohol withdrawl',
  'C)Alcohol abstinence',
  'D)Alcohol intoxication'],
 'prompt': '\n\nQ:‘Blackouts’ in a person of alcohol consumption is seen in which of the following conditions?A)Alcohol dependence B)Alcohol withdrawl C)Alcohol abstinence D)Alcohol intoxication\n\nA:',
 'output1': '\n<bot>: Because blackouts are a common symptom of alcohol dependence, which is characterized by a pattern of continued drinking despite negative consequences, as well as withdrawal symptoms when alcohol use is stopped.\n\n',
 'output2': '\n<bot>: When a person consumes alcohol regularly and then suddenly stops or reduces their consumption, they may experience withdrawal symptoms such as anxiety, tremors, and seizures. These symptoms can be severe and can lead to blackouts. Alcohol withdrawal can be a dangerou

In [ ]:
def update_max_length(i, j, max_len, max_output_len):
    current_len = len(i["output" + str(j)])
    if current_len > max_len:
        return current_len, "output" + str(j)
    return max_len, max_output_len

def update_rouge_scores(i, j, min_rouge_1_recall, min_output_rouge_1_recall, min_rouge_L_recall, min_output_rouge_L_recall):
    base_text = i["question"] + i["output"+str(j)]
    rouge_1_recall = rouge.get_scores(i["output" + str(j)], base_text)[0]['rouge-1']['r']
    rouge_L_recall = rouge.get_scores(i["output" + str(j)], base_text)[0]['rouge-l']['r']

    if rouge_1_recall < min_rouge_1_recall:
        min_rouge_1_recall, min_output_rouge_1_recall = rouge_1_recall, "output" + str(j)

    if rouge_L_recall < min_rouge_L_recall:
        min_rouge_L_recall, min_output_rouge_L_recall = rouge_L_recall, "output" + str(j)

    return min_rouge_1_recall, min_output_rouge_1_recall, min_rouge_L_recall, min_output_rouge_L_recall

def update_nsp_score(i, j, max_nsp_score, max_output_nsp):
    base_text = i["question"] + i["output"+str(j)]
    nsp_score = nsp(base_text, i["output" + str(j)])
    if nsp_score > max_nsp_score:
        return nsp_score, "output" + str(j)
    return max_nsp_score, max_output_nsp

def update_cosine_similarity(i, j, min_cosine_score, min_output_cosine):
    base_text = i["question"] + i["output"+str(j)]
    cosine_score = cosine(i["output" + str(j)], base_text)
    if cosine_score < min_cosine_score:
        return cosine_score, "output" + str(j)
    return min_cosine_score, min_output_cosine


In [ ]:
temp = data[0:1]

In [ ]:
pbar = tqdm.tqdm(temp)
for i in pbar:
    # Initialize variables
    max_len, max_output_len = -1*float('inf'), None
    min_rouge_1_recall, min_output_rouge_1_recall = float('inf'), None
    min_rouge_L_recall, min_output_rouge_L_recall = float('inf'), None
    max_nsp_score, max_output_nsp = -1, None
    min_cosine_score, min_output_cosine = float('inf'), None

    correct_idx = [op.split(")")[1] for op in i["options"]].index(i["correct_answer"])
    correct_op = correct_idx + 1

    for j in range(1, 5):
        if j == correct_op:
            continue

        # Update scores using functions
        max_len, max_output_len = update_max_length(i, j, max_len, max_output_len)
        min_rouge_1_recall, min_output_rouge_1_recall, min_rouge_L_recall, min_output_rouge_L_recall = update_rouge_scores(i, j, min_rouge_1_recall, min_output_rouge_1_recall, min_rouge_L_recall, min_output_rouge_L_recall)
        max_nsp_score, max_output_nsp = update_nsp_score(i, j, max_nsp_score, max_output_nsp)
        min_cosine_score, min_output_cosine = update_cosine_similarity(i, j, min_cosine_score, min_output_cosine)

    # Store the results in i
    i['lowest_rouge-1'] = min_output_rouge_1_recall
    i['lowest_rouge-L'] = min_output_rouge_L_recall
    i['max_len'] = max_output_len
    i['min_cosine'] = min_output_cosine


100%|██████████| 1/1 [00:03<00:00,  3.57s/it]


In [ ]:
data[0]

{'question': 'Chordoma arises from:',
 'correct_answer': 'Notochord',
 'options': ['A)Notochord',
  "B)Luschka's bursa",
  'C)Pharyngeal bursa',
  "D)Rathke's pouch"],
 'prompt': "\n\nQ:Chordoma arises from:A)Notochord B)Luschka's bursa C)Pharyngeal bursa D)Rathke's pouch\n\nA:",
 'output1': '\n<bot>: Chordoma arises from the remnants of the notochord, a structure that is present in the embryo and then disappears.\n\n',
 'output2': "\n<bot>: Because chordoma arises from the remnants of Luschka's bursa, which is a type of membranous structure that forms in the pharynx during embryonic development.",
 'output3': '\n<bot>: Because chordoma arises from the pharyngeal bursa, which is a remnant of the pharyngeal arch.\n<human>',
 'output4': "\n<bot>: Because chordoma arises from a type of cell called a chordoma cell, which is found in Rathke's pouch. Rathke's pouch is a structure that is present in the neck and is thought to be the site of origin of chordoma.\n\n",
 'lowest_rouge-1': 'output

In [ ]:
temp

[{'question': 'Chordoma arises from:',
  'correct_answer': 'Notochord',
  'options': ['A)Notochord',
   "B)Luschka's bursa",
   'C)Pharyngeal bursa',
   "D)Rathke's pouch"],
  'prompt': "\n\nQ:Chordoma arises from:A)Notochord B)Luschka's bursa C)Pharyngeal bursa D)Rathke's pouch\n\nA:",
  'output1': '\n<bot>: Chordoma arises from the remnants of the notochord, a structure that is present in the embryo and then disappears.\n\n',
  'output2': "\n<bot>: Because chordoma arises from the remnants of Luschka's bursa, which is a type of membranous structure that forms in the pharynx during embryonic development.",
  'output3': '\n<bot>: Because chordoma arises from the pharyngeal bursa, which is a remnant of the pharyngeal arch.\n<human>',
  'output4': "\n<bot>: Because chordoma arises from a type of cell called a chordoma cell, which is found in Rathke's pouch. Rathke's pouch is a structure that is present in the neck and is thought to be the site of origin of chordoma.\n\n",
  'lowest_rouge

In [ ]:
import pandas as pd
with open("select_option.json", "w") as file:
  json.dump(data[0:200], file)

In [ ]:
# df = pd.read_json('select_option.json')
# df.to_csv('select_option.csv', index=False)

### Analyzing the different metrics

In [ ]:
import chardet

# Detect the encoding
with open('select_option.csv', 'rb') as file:
    result = chardet.detect(file.read())
    encoding = result['encoding']

# Read the CSV file using the detected encoding
metrics = pd.read_csv("select_option.csv", encoding=encoding)


In [ ]:
metrics.head()

,question,correct_answer,options,prompt,output1,output2,output3,output4,lowest_rouge-1,lowest_rouge-L,max_len,min_cosine,max_nsp,output1_ann,output2_ann,output3_ann,output4_ann
0,Chordoma arises from:,Notochord,"['A)Notochord', ""B)Luschka's bursa"", 'C)Pharyn...",\n\nQ:Chordoma arises from:A)Notochord B)Lusch...,\n<bot>: Chordoma arises from the remnants of ...,\n<bot>: Because chordoma arises from the remn...,\n<bot>: Because chordoma arises from the phar...,\n<bot>: Because chordoma arises from a type o...,output3,output3,output4,output3,output3,NaN,NaN,NaN,1.0
1,Among the models of Item Response Theory (TRI)...,The parameter a is constant and the parameter ...,['A)The parameter a is constant and the parame...,\n\nQ:Among the models of Item Response Theory...,\n<bot>: Because the logistic model assumes th...,\n<bot> A: Because the logistic model assumes ...,\n<bot> A: Because the logistic model assumes ...,\n<bot> A: Because the logistic model assumes ...,output4,output4,output1,output2,output4,NaN,NaN,NaN,NaN
2,A 54-year-old woman is brought to the physicia...,Intravenous thiamine,"['A)Intravenous vitamin B12', 'B)Intravenous g...",\n\nQ:A 54-year-old woman is brought to the ph...,\n<bot> A: The patient's symptoms of confusion...,\n<bot> A: The patient's symptoms of confusion...,\n<bot> A: The patient's symptoms of confusion...,\n<bot> A: The patient's symptoms of confusion...,output1,output1,output2,output3,output1,1.0,NaN,NaN,NaN
3,ýýýBlackoutsýýý in a person of alcohol consump...,Alcohol intoxication,"['A)Alcohol dependence', 'B)Alcohol withdrawl'...",\n\nQ:ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½Blackoutsï¿½ï...,\n<bot>: Because blackouts are a common sympto...,\n<bot>: When a person consumes alcohol regula...,\n<bot>: When a person who is alcohol dependen...,"\n<bot>: When a person consumes alcohol, it ca...",output1,output1,output2,output1,output2,NaN,1.0,1.0,NaN
4,Isoniazid (isonicotinic acid hydrazide; isonia...,Effect of cell mycolic acids (mycolic acid) ge...,['A)Inhibition of bacterial protein synthesis'...,\n\nQ:Isoniazid (isonicotinic acid hydrazide; ...,\n<bot>: Isoniazid inhibits the synthesis of b...,\n<bot>: Isoniazid inhibits the synthesis of n...,\n\n,\n\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
metrics.columns

Index(['question', 'correct_answer', 'options', 'prompt', 'output1', 'output2',
       'output3', 'output4', 'lowest_rouge-1', 'lowest_rouge-L', 'max_len',
       'min_cosine', 'max_nsp', 'output1_ann', 'output2_ann', 'output3_ann',
       'output4_ann'],
      dtype='object')

In [ ]:
import numpy as np
metrics["output1_ann"][2] != 1.

False

In [ ]:
metric_cols = ['lowest_rouge-1', 'lowest_rouge-L', 'max_len','min_cosine', 'max_nsp']

In [ ]:
num = 0
den = 0
check_cols = ["output1_ann", 	"output2_ann", 	"output3_ann"	, "output4_ann"]
for k in metric_cols:
  k_idx = list(metrics.columns).index(k)
  for i in range(len(metrics)):
    if list(metrics[check_cols].sum(axis = 1))[i]==0:
      continue
    else:
      den = den + 1
      check = metrics.iloc[i, k_idx]
      if list(metrics[check+"_ann"])[i] == 1:
        num = num + 1
  print (k, num/den*100)

lowest_rouge-1 32.142857142857146
lowest_rouge-L 32.142857142857146
max_len 51.19047619047619
min_cosine 43.75
max_nsp 50.0


### Modifying the data

In [ ]:
with open("medhalt_corrected.json", "r") as file:
  data = json.load(file)

In [ ]:
new_data = []
pbar = tqdm.tqdm(data)
for i in pbar:
  try:
    max_len, max_output_len = -1*float('inf'), None
    correct_idx = [op.split(")", 1)[1] for op in i["options"]].index(i["correct_answer"])
    correct_op = correct_idx + 1

    for j in range(1, 5):
        if j == correct_op:
            continue
        # Update scores using functions
        max_len, max_output_len = update_max_length(i, j, max_len, max_output_len)
    i["correct_idx"] = correct_op
    i['max_len'] = max_output_len
    new_data.append(i)
  except:
    print (str(i))


100%|██████████| 16971/16971 [00:00<00:00, 149052.86it/s]

{'question': 'Peripheral eosinophila with renal failure is least likely to be possible in:', 'correct_answer': 'Polyangitis nodosa', 'options': ['A)Polyangitis nodosa', 'B)Atheroembolic renal failure', 'C)Drug-induced interstitial nephritis'], 'prompt': '\n\nQ:Peripheral eosinophila with renal failure is least likely to be possible in:A)Polyangitis nodosa B)Atheroembolic renal failure C)Drug-induced interstitial nephritis\n\nA:'}
{'question': 'Not increased in stress ?', 'options': ['A)GH', 'B)None', 'C)ADH', 'D)thyroxine'], 'prompt': '\n\nQ:Not increased in stress ?A)GH B)None C)ADH D)thyroxine\n\nA:', 'output1': '\n\n', 'output2': "\n<bot>: Because the body's response to stress is to release hormones such as adrenaline and cortisol, which can help to increase alertness and energy. However, these hormones do not increase GH levels. In fact, high levels of cortisol can actually decrease GH levels.\n\n", 'output3': '\n<bot>: ADH is anti-diuretic hormone, which helps to increase the amou

In [ ]:
len(new_data)

16921

In [ ]:
new_data[0]

{'question': 'Chordoma arises from:',
 'correct_answer': 'Notochord',
 'options': ['A)Notochord',
  "B)Luschka's bursa",
  'C)Pharyngeal bursa',
  "D)Rathke's pouch"],
 'prompt': "\n\nQ:Chordoma arises from:A)Notochord B)Luschka's bursa C)Pharyngeal bursa D)Rathke's pouch\n\nA:",
 'output1': '\n<bot>: Chordoma arises from the remnants of the notochord, a structure that is present in the embryo and then disappears.\n\n',
 'output2': "\n<bot>: Because chordoma arises from the remnants of Luschka's bursa, which is a type of membranous structure that forms in the pharynx during embryonic development.",
 'output3': '\n<bot>: Because chordoma arises from the pharyngeal bursa, which is a remnant of the pharyngeal arch.\n<human>',
 'output4': "\n<bot>: Because chordoma arises from a type of cell called a chordoma cell, which is found in Rathke's pouch. Rathke's pouch is a structure that is present in the neck and is thought to be the site of origin of chordoma.\n\n",
 'max_len': 'output4',
 'c

In [ ]:
with open("corrected_medhalt_max_len_bad_op.json", "w") as file:
  json.dump(new_data, file)

### standardizing generations

In [ ]:
# standardize correct op and max_len op
pbar = tqdm.tqdm(new_data)
std_data = []
for i in pbar:
  prompt = "\n\n####user\n: " + i["question"] + " ".join(i["options"])
  cor_gen = "output" + str(i["correct_idx"])
  cor_gen = i[cor_gen]
  cor_gen = cor_gen.replace("<human>", "").replace("\n", "").replace("\n<bot>:", "\n####assistant\n").replace("<bot>", "\n####assistant\n")

  op2 = i["max_len"]
  op2_gen = i[op2]
  op2_gen = op2_gen.replace("<human>", "").replace("\n", "").replace("\n<bot>:", "\n####assistant\n").replace("<bot>", "\n####assistant\n")

  if "\n####assistant\n" in cor_gen:
    i["output" + str(i["correct_idx"])] = cor_gen
    i[op2] = op2_gen
    std_data.append(i)
  else:
    continue

100%|██████████| 16921/16921 [00:00<00:00, 82592.50it/s]


In [ ]:
with open("zero_shot_medhalt.json", "w") as file:
  json.dump(std_data, file)

In [ ]:
# basic checks what is the len of the correct option and the ideal bad option
# also all A) B) C) D) should not be mentioned in
reject_list = []
approve_list = []
pbar = tqdm.tqdm(std_data)
for i in pbar:
  rej_op = i["output" + str(i["correct_idx"])]
  sel_op = i[i["max_len"]]

  if int("A)" in rej_op) + int("B)" in rej_op) + int("C)" in rej_op) + int("D)" in rej_op)>1:
    reject_list.append(i)
  elif int("A)" in sel_op) + int("B)" in sel_op) + int("C)" in sel_op) + int("D)" in sel_op)>1:
    reject_list.append(i)

  elif len(rej_op)<10 or len(sel_op)<10:
    reject_list.append(i)
  else:
    approve_list.append(i)

100%|██████████| 15699/15699 [00:00<00:00, 110336.66it/s]


In [ ]:
len(reject_list)

579

In [ ]:
len(approve_list)

15120

In [ ]:
with open("zero_shot_medhalt_cleaned.json", "w") as file:
  json.dump(approve_list, file)